In [117]:
#Importing packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#Setting large figure size for Seaborn
sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":18})

#Importing Intel extension for sklearn to improve speed
# from sklearnex import patch_sklearn
# patch_sklearn()

In [118]:
orders = pd.read_csv("Data/olist_orders_dataset.csv")
order_items = pd.read_csv("Data/olist_order_items_dataset.csv")
order_payments = pd.read_csv("Data/olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("Data/olist_order_reviews_dataset.csv")
products = pd.read_csv("Data/olist_products_dataset.csv")
sellers = pd.read_csv("Data/olist_sellers_dataset.csv")
customers = pd.read_csv("Data/olist_customers_dataset.csv")
locations = pd.read_csv("Data/olist_geolocation_dataset.csv")
category_translation = pd.read_csv("Data/product_category_name_translation.csv")

order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [119]:
#Importing the itertools combinations function that will generate unique pairs of our list
from itertools import combinations
#Importing Levenshtein, an algorithm that calculates the distance between the strings
from Levenshtein import distance as levenshtein_distance

#Defining our distance function
def calculate_lev_dist(x):
    return levenshtein_distance(x['first'],x['second'])

#Defining the function that will iterate over the specified indexes of our dataframe and create
#A list of dataframes containing all the pairs ordered by Levenshtein distance (ascending)
def check_similarities(df: pd.DataFrame, index):
    i = 0
    dataframes = []
    for c in df.columns:
        names = []
        if np.isin(i, index):
            #Creating a list of unique names
            name_list = df[c].dropna().unique()
            #Creating an empty array
            names = []
            #Iterating over all pairs of our list
            for c in combinations(name_list,2):
                #Appending a dictionary to the list with the 2 elements
                names.append({'first': c[0], 'second': c[1]})
            #Creating a names_df dataframe
            names_df = pd.DataFrame(names)
            #Calculating the levenshtein distance for each value pairs
            names_df['lev_dist'] = names_df.apply(calculate_lev_dist, axis=1)
            #Sorting the dataframe by levenshtein distance
            names_df.sort_values(by='lev_dist', inplace=True)
            #Adding our names_df to our list of dataframes (called 'dataframes')
            dataframes.append(names_df)
        i += 1
    return dataframes

check_similarities(products, [1])[0]

products.loc[products.product_category_name == 'eletrodomesticos_2', 'product_category_name'] = "eletrodomesticos"
products.loc[products.product_category_name == 'casa_conforto_2', 'product_category_name'] = "casa_conforto"
products.loc[products.product_category_name == 'pc_gamer', 'product_category_name'] = "pcs"

products.product_category_name.value_counts()

cama_mesa_banho                  3029
esporte_lazer                    2867
moveis_decoracao                 2657
beleza_saude                     2444
utilidades_domesticas            2335
                                 ... 
moveis_colchao_e_estofado          10
tablets_impressao_imagem            9
fashion_roupa_infanto_juvenil       5
seguros_e_servicos                  2
cds_dvds_musicais                   1
Name: product_category_name, Length: 70, dtype: int64

In [120]:
#Figuring out if all translated names are in the translation
[x for x in products.product_category_name.unique() if not np.isin(x, category_translation.product_category_name.unique())]

[nan, 'portateis_cozinha_e_preparadores_de_alimentos']

In [121]:
#Manually adding not translated category
category_translation = category_translation.append({'product_category_name': 'portateis_cozinha_e_preparadores_de_alimentos',
                             'product_category_name_english': 'kitchen_and_food_preparation_appliances'}, ignore_index=True)

products = pd.merge(products, category_translation, on='product_category_name', how='inner')

products.drop(columns={'product_category_name'}, inplace=True)
products.rename(columns={'product_category_name_english': 'product_category_name'}, inplace=True)
products.head()


C:\Users\fatsk\AppData\Local\Temp\ipykernel_26900\414929226.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_translation = category_translation.append({'product_category_name': 'portateis_cozinha_e_preparadores_de_alimentos',


,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery


In [122]:
#Inner join because we will eventually drop the products that have no info
order_products = pd.merge(order_items, products, on='product_id', how='inner')
#Dropping the product name and description length that will not be relevant
order_products = order_products.drop(columns={"product_name_lenght","product_description_lenght","product_id"})
#We also remove product_id since we have already gathered all the data from the product_id table
order_products.head()

,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,4.0,650.0,28.0,9.0,14.0,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,4.0,650.0,28.0,9.0,14.0,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,4.0,650.0,28.0,9.0,14.0,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,1,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,4.0,650.0,28.0,9.0,14.0,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,1,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,4.0,650.0,28.0,9.0,14.0,cool_stuff


In [123]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [124]:
locations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [125]:
#Several locations for each zip_code_prefix
locations.groupby(["geolocation_zip_code_prefix","geolocation_city","geolocation_state"]).nunique()

#We will average them to have a mean location for each zip_code to prepare the merge with our sellers dataframe

locations_zip = locations[["geolocation_zip_code_prefix",
                       "geolocation_lat", "geolocation_lng"]].groupby("geolocation_zip_code_prefix").mean().reset_index()

locations_zip

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.550190,-46.634024
1,1002,-23.548146,-46.634979
2,1003,-23.548994,-46.635731
3,1004,-23.549799,-46.634757
4,1005,-23.549456,-46.636733
...,...,...,...
19010,99960,-27.953722,-52.025511
19011,99965,-28.183372,-52.039850
19012,99970,-28.343766,-51.874689
19013,99980,-28.389129,-51.843836


In [126]:
sellers = pd.merge(sellers.drop(columns={"seller_city"}), locations_zip, how="left",
                   left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix")

sellers.drop(columns={"geolocation_zip_code_prefix"}, inplace=True)
sellers.rename(columns={"geolocation_lat": "seller_lat", "geolocation_lng": "seller_long"},inplace=True)

sellers.head()

,seller_id,seller_zip_code_prefix,seller_state,seller_lat,seller_long
0,3442f8959a84dea7ee197c632cb2df15,13023,SP,-22.893848,-47.061337
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,SP,-22.383437,-46.947927
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,RJ,-22.909572,-43.177703
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,SP,-23.657242,-46.612831
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,SP,-22.964803,-46.534419


In [127]:
sellers[sellers.seller_lat.isna()]
#8 sellers without geolocation

#Averaging the location of all states to gather data from sellers with no matching zip code
locations_state = locations[["geolocation_state",
                       "geolocation_lat", "geolocation_lng"]].groupby("geolocation_state").mean().reset_index()

missing_loc_sellers = pd.merge(sellers[sellers.seller_lat.isna()].drop(columns={"seller_lat","seller_long"}),
                               locations_state, how="left", left_on="seller_state", right_on="geolocation_state")

missing_loc_sellers.drop(columns={"geolocation_state"}, inplace=True)
missing_loc_sellers.rename(columns={"geolocation_lat": "seller_lat", "geolocation_lng": "seller_long"},inplace=True)

missing_loc_sellers

,seller_id,seller_zip_code_prefix,seller_state,seller_lat,seller_long
0,5962468f885ea01a1b6a97a218797b0a,82040,PR,-24.793607,-50.879662
1,2aafae69bf4c41fbd94053d9413e87ee,91901,RS,-29.679191,-52.032652
2,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,DF,-15.810885,-47.969630
3,2e90cb1677d35cfe24eef47d441b7c87,2285,SP,-23.155308,-47.084074
4,0b3f27369a4d8df98f7eb91077e438ac,7412,SP,-23.155308,-47.084074
5,42bde9fef835393bb8a8849cb6b7f245,71551,DF,-15.810885,-47.969630
6,870d0118f7a9d85960f29ad89d5d989a,37708,MG,-19.864647,-44.421615


In [128]:
sellers = sellers[sellers.seller_lat.notna()]

sellers = pd.concat([sellers, missing_loc_sellers])

sellers[sellers.seller_lat.isna()]
#No missing coordinate

,seller_id,seller_zip_code_prefix,seller_state,seller_lat,seller_long


In [129]:
#Mergin seller info back into our order_products dataframe

order_products = pd.merge(order_products, sellers, how="left", on="seller_id" )

order_products[order_products.seller_lat.isna()]
#All items were merged

order_products.drop(columns={"seller_id"}, inplace=True)

order_products.duplicated().any()
#No duplicates

order_products

,order_id,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long
0,00010242fe8c5a6d1ba2dd792cb16214,1,2017-09-19 09:45:35,58.9,13.29,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,SP,-22.496953,-44.127492
1,130898c0987d1801452a8ed92a670612,1,2017-07-05 02:44:11,55.9,17.96,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,SP,-22.496953,-44.127492
2,532ed5e14e24ae1f0d735b91524b98b9,1,2018-05-23 10:56:25,64.9,18.33,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,SP,-22.496953,-44.127492
3,6f8c31653edb8c83e1a739408b5ff750,1,2017-08-07 18:55:08,58.9,16.17,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,SP,-22.496953,-44.127492
4,7d19f4ef4d04461989632411b7e588b9,1,2017-08-16 22:05:11,58.9,13.29,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,SP,-22.496953,-44.127492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111042,ffebd80e3291e811c308365936897efd,1,2018-07-18 20:23:55,129.9,51.20,1.0,6700.0,35.0,12.0,22.0,garden_tools,99730,RS,-27.728956,-52.536535
111043,ffee31fb4b5e35c9123608015637c495,1,2018-08-21 10:10:11,99.0,13.52,1.0,2300.0,37.0,30.0,20.0,furniture_decor,80330,PR,-25.469974,-49.307372
111044,fff7c4452f050315db1b3f24d9df5fcd,1,2017-06-07 17:05:23,736.0,20.91,3.0,400.0,19.0,9.0,15.0,watches_gifts,6429,SP,-23.489688,-46.952915
111045,fffa82886406ccf10c7b4e35c4ff2788,1,2017-12-22 17:31:42,229.9,44.02,2.0,2700.0,60.0,15.0,15.0,sports_leisure,5141,SP,-23.492024,-46.728984


In [130]:
#Order ids without product are discarded, we do an inner join here
orders = pd.merge(orders, order_products, how="inner", on="order_id")

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,2017-10-06 11:07:15,29.99,8.72,4.0,500.0,19.0,8.0,13.0,housewares,9350,SP,-23.680729,-46.444238
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,2018-07-30 03:24:27,118.70,22.76,1.0,400.0,19.0,13.0,19.0,perfumery,31570,SP,-19.807681,-43.980427
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,2018-08-13 08:55:23,159.90,19.22,1.0,420.0,24.0,19.0,21.0,auto,14840,SP,-21.363502,-48.229601
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,2017-11-23 19:45:59,45.00,27.20,3.0,450.0,30.0,10.0,20.0,pet_shop,31842,MG,-19.837682,-43.924053
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,2018-02-19 20:31:37,19.90,8.72,4.0,250.0,51.0,15.0,15.0,stationery,8752,SP,-23.543395,-46.262086


In [131]:
order_payments.head()

print(order_payments.order_id.nunique(), len(order_payments))


99440 103886


In [132]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [133]:
unique_customers = customers.groupby("customer_id").agg(unique_ids= ('customer_id', 'nunique')).reset_index()

unique_customers[unique_customers.unique_ids > 1]
#Returns null dataframe, all customer id are unique

del unique_customers

#We can delete the customer_unique_id
customers.drop(columns={"customer_unique_id"}, inplace=True)

customers = pd.merge(customers, locations_zip, how="left",
                     left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix")

print(customers[customers.geolocation_lat.isna()])
customers.head()

                            customer_id  customer_zip_code_prefix  \
354    ecb1725b26e8b8c458181455dfa434ea                     72300   
382    bcf86029aeed4ed8bac0e16eb14c22f5                     11547   
877    f4302056f0c58570522590f8181de2c7                     64605   
1218   03bbe0ce5c28e05f22917607db798818                     72465   
1272   ad4950aded55c2ea376be59506456d68                      7729   
...                                 ...                       ...   
97467  cf818420383856a129134f5f8343f7b8                     72338   
97780  67f3e907dce402e696b15f9308ff22ed                     68629   
98140  f792e419335df11d82c32efcfb09c51b                     28530   
98878  78a11bb1fa72f556996b9a5b9bcd0629                     55863   
99264  ff09fd7b29e7488a8d8a20badcd8befe                     72587   

                  customer_city customer_state  geolocation_zip_code_prefix  \
354                    brasilia             DF                          NaN   
382          

,customer_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,14409.0,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,9790.0,-23.727992,-46.542848
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,1151.0,-23.531642,-46.656289
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,8775.0,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,13056.0,-22.975100,-47.142925


In [134]:
#As with sellers, we replace the missing geocordinate data by the mean data by state
missing_cust_geo = pd.merge(customers[customers.geolocation_lat.isna()].drop(columns={"geolocation_lat","geolocation_lng"}),
                            locations_state, how="left", left_on="customer_state", right_on="geolocation_state")

print(missing_cust_geo[missing_cust_geo.geolocation_lat.isna()])
#No more missing geocordinates

missing_cust_geo.drop(columns={"geolocation_zip_code_prefix","geolocation_state"}, inplace=True)
missing_cust_geo.head()

Empty DataFrame
Columns: [customer_id, customer_zip_code_prefix, customer_city, customer_state, geolocation_zip_code_prefix, geolocation_state, geolocation_lat, geolocation_lng]
Index: []


,customer_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng
0,ecb1725b26e8b8c458181455dfa434ea,72300,brasilia,DF,-15.810885,-47.969630
1,bcf86029aeed4ed8bac0e16eb14c22f5,11547,cubatao,SP,-23.155308,-47.084074
2,f4302056f0c58570522590f8181de2c7,64605,picos,PI,-5.754989,-42.509541
3,03bbe0ce5c28e05f22917607db798818,72465,brasilia,DF,-15.810885,-47.969630
4,ad4950aded55c2ea376be59506456d68,7729,caieiras,SP,-23.155308,-47.084074


In [135]:
customers = customers[customers.geolocation_lat.notna()]

customers = pd.concat([customers, missing_cust_geo], ignore_index=True)

customers.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,14409.0,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,9790.0,-23.727992,-46.542848
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,1151.0,-23.531642,-46.656289
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,8775.0,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,13056.0,-22.975100,-47.142925


In [136]:
customers.drop(columns={"geolocation_zip_code_prefix"}, inplace=True)
customers.rename(columns={"geolocation_lat": "customer_lat", "geolocation_lng": "customer_long"}, inplace=True)

customers.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925


In [137]:
#Merging on customers since we want to study customers and want to drop orders without customers
df = pd.merge(customers, orders, how="left", on="customer_id")

print(len(df.customer_id.unique()), len(df.order_id.unique()), len(df))

99441 97278 113211


In [138]:
order_reviews.head()

order_reviews[order_reviews["order_id"].duplicated()]
#Some orders have different review

order_reviews["review_creation_date"] = pd.to_datetime(order_reviews["review_creation_date"])

#Aggregating by order id, averaging review score and review date and couting the number of columns and messages/titles
order_reviews = order_reviews.groupby("order_id").agg(
    review_score = ('review_score' , 'mean'), review_date = ('review_creation_date', 'mean'),
    review_ncomments = ('review_comment_message','count'), review_ntitle = ('review_comment_title', 'count'),
    n_reviews = ('review_id', 'nunique')).reset_index()

order_reviews.head()

,order_id,review_score,review_date,review_ncomments,review_ntitle,n_reviews
0,00010242fe8c5a6d1ba2dd792cb16214,5.0,2017-09-21,1,0,1
1,00018f77f2f0320c557190d7a144bdd3,4.0,2017-05-13,0,0,1
2,000229ec398224ef6ca0657da4fc703e,5.0,2018-01-23,1,0,1
3,00024acbcdf0a6daa1e931b038114c75,4.0,2018-08-15,0,0,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0,2017-03-02,1,0,1


In [139]:
df = pd.merge(df, order_reviews, how="left", on="order_id")

df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,8683.0,54.0,64.0,31.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,10150.0,89.0,15.0,40.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,8267.0,52.0,52.0,17.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,12160.0,56.0,51.0,28.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,5200.0,45.0,15.0,35.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0


In [140]:
order_payments["order_id"].duplicated().any()
#Some duplicate orders in this dataset

order_payments.head()

order_payments_type = pd.crosstab(index=order_payments["order_id"], columns=order_payments["payment_type"])

order_payments_type

payment_type,boleto,credit_card,debit_card,not_defined,voucher
order_id,,,,,
00010242fe8c5a6d1ba2dd792cb16214,0,1,0,0,0
00018f77f2f0320c557190d7a144bdd3,0,1,0,0,0
000229ec398224ef6ca0657da4fc703e,0,1,0,0,0
00024acbcdf0a6daa1e931b038114c75,0,1,0,0,0
00042b26cf59d7ce69dfabb4e55b4fd9,0,1,0,0,0
...,...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,1,0,0,0,0
fffcd46ef2263f404302a634eb57f7eb,1,0,0,0,0
fffce4705a9662cd70adb13d4a31832d,0,1,0,0,0


In [141]:
order_payments_agg = order_payments.groupby("order_id").agg(
    payment_sequential = ('payment_sequential', 'mean'), payment_installments = ('payment_installments', 'sum'),
    payment_value = ('payment_value', 'sum')).reset_index()

#Flagging order with sequential payments
order_payments_agg["payment_sequential_flag"] = 0
order_payments_agg.loc[order_payments_agg.payment_sequential > 1] = 1

order_payments_agg.head()

,order_id,payment_sequential,payment_installments,payment_value,payment_sequential_flag
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,2,72.19,0
1,00018f77f2f0320c557190d7a144bdd3,1.0,3,259.83,0
2,000229ec398224ef6ca0657da4fc703e,1.0,5,216.87,0
3,00024acbcdf0a6daa1e931b038114c75,1.0,2,25.78,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,3,218.04,0


In [142]:
payments = pd.merge(order_payments_agg, order_payments_type, how="inner", on="order_id")

df = pd.merge(df, payments, how="left", on="order_id")

df

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,8683.0,54.0,64.0,31.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,10150.0,89.0,15.0,40.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,8267.0,52.0,52.0,17.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,12160.0,56.0,51.0,28.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,5200.0,45.0,15.0,35.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113206,cf818420383856a129134f5f8343f7b8,72338,brasilia,DF,-15.810885,-47.969630,e13f9d9f79f8325d4f77d26e7384e9fd,delivered,2017-07-11 18:23:43,2017-07-13 03:04:39,2017-07-14 18:34:26,2017-07-25 02:53:28,2017-08-04 00:00:00,1.0,2017-07-19 03:04:39,44.90,14.10,4.0,200.0,16.0,7.0,11.0,toys,2310.0,SP,-23.472815,-46.600729,5.0,2017-07-25,1.0,0.0,1.0,1.0,1.0,59.00,0.0,1.0,0.0,0.0,0.0,0.0
113207,67f3e907dce402e696b15f9308ff22ed,68629,paragominas,PA,-2.631213,-49.485862,e2f630b53cb2637be372c3810b089868,shipped,2017-07-26 11:44:28,2017-07-27 11:25:25,2017-07-27 17:47:51,NaN,2017-08-29 00:00:00,1.0,2017-08-02 11:25:25,149.00,26.32,7.0,250.0,16.0,2.0,11.0,watches_gifts,5849.0,SP,-23.652366,-46.755753,2.0,2017-09-01,1.0,0.0,1.0,1.0,1.0,175.32,0.0,1.0,0.0,0.0,0.0,0.0
113208,f792e419335df11d82c32efcfb09c51b,28530,sao sebastiao do paraiba,RJ,-22.743477,-43.155540,84a80b02b3af075990fc7d9d2369f066,delivered,2017-10-15 19:33:54,2017-10-18 08:35:17,2017-10-19 22:27:47,2017-11-03 11:08:03,2017-11-10 00:00:00,1.0,2017-10-24 08:35:17,540.00,31.16,1.0,3375.0,47.0,22.0,11.0,auto,79400.0,MS,-18.507265,-54.748753,5.0,2017-11-04,1.0,0.0,1.0,1.0,5.0,571.16,0.0,0.0,1.0,0.0,0.0,0.0
11320

In [143]:
#Dropping columns with no order_id
df = df[df.order_id.notna()]

print(df.order_status.unique())

#Keeping only completed or cancelled orders
df = df[(df.order_status=="delivered") | (df.order_status=="canceled")]

df[df.payment_value.isna()]

['delivered' 'invoiced' 'shipped' 'processing' 'canceled' 'unavailable'
 'approved']


,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher
18,9b8ce803689b3562defaad4613ef426f,5416,sao paulo,SP,-23.560949,-46.688422,17825f24877a9289214c301ae0c9424b,delivered,2017-05-11 13:48:47,2017-05-13 11:55:16,2017-05-15 15:30:02,2017-06-07 12:02:14,2017-05-30 00:00:00,1.0,2017-05-19 11:55:16,89.90,12.13,1.0,1383.0,50.0,10.0,40.0,bed_bath_table,14940.0,SP,-21.757321,-48.829744,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,469634941c27cd844170935a3cf60b95,81750,curitiba,PR,-25.517959,-49.241322,a9119eb77d6200811953803a7b6539e1,delivered,2018-03-12 13:07:03,2018-03-12 13:15:28,2018-03-13 22:12:18,2018-03-22 18:28:55,2018-03-29 00:00:00,1.0,2018-03-16 13:15:28,30.50,15.23,3.0,200.0,16.0,8.0,11.0,toys,2310.0,SP,-23.472815,-46.600729,5.0,2018-03-23,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,8247b5583327ab8be19f96e1fb82f77b,23970,parati,RJ,-23.212886,-44.707780,a6917b5d71e0e9bc434e9228db8daeb2,delivered,2017-06-09 15:46:17,2017-06-10 15:42:38,2017-06-12 17:10:55,2017-06-19 19:32:32,2017-07-04 00:00:00,1.0,2017-06-15 15:42:38,788.42,21.28,1.0,450.0,18.0,11.0,14.0,watches_gifts,6429.0,SP,-23.489688,-46.952915,1.0,2017-06-22,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,a02f66c3af7b16eec19ddcd98b645fe3,13323,salto,SP,-23.216315,-47.271594,db97652cf517d2cd03db63dec489ca62,delivered,2017-10-01 08:57:03,2017-10-01 09:14:07,2017-10-02 19:32:57,2017-10-03 14:23:50,2017-10-18 00:00:00,1.0,2017-10-05 09:14:07,19.90,7.78,1.0,350.0,27.0,5.0,20.0,computers_accessories,1026.0,SP,-23.539657,-46.631884,3.0,2017-10-04,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,91ec76836092bba85d11761078ed7bb5,28010,campos dos goytacazes,RJ,-21.761750,-41.322672,9cefab6270eb935eb96a97c56b8e7984,delivered,2018-06-07 21:41:19,2018-06-07 21:54:43,2018-06-11 12:16:00,2018-06-15 17:28:42,2018-07-05 00:00:00,1.0,2018-06-17 21:54:43,330.00,20.19,6.0,400.0,34.0,4.0,22.0,health_beauty,9628.0,SP,-23.661305,-46.564296,4.0,2018-06-16,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113065,e198d9344eb2d92c850249a49d874dbb,72238,brasilia,DF,-15.810885,-47.969630,2153e9009ecf443d84e5f48861fb7094,delivered,2017-12-06 16:02:30,2017-12-08 19:35:22,2017-12-12 20:15:37,2017-12-26 19:46:50,2018-01-02 00:00:00,2.0,2017-12-14 19:35:22,39.99,14.10,7.0,150.0,19.0,3.0,11.0,telephony,8250.0,SP,-23.539944,-46.439816,5.0,2017-12-27,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113114,8a8476a80099f2b63c573d23679532ec,94370,ipiranga,RS,-29.679191,-52.032652,7e8072dc0f35ebb0c1b2a4743e0f179a,delivered,2018-06-27 14:07:03,2018-06-28 20:10:14,2018-06-29 08:25:00,2018-07-11 00:12:50,2018-07-27 00:00:00,1.0,2018-07-04 20:10:14,79.00,16.74,1.0,750.0,25.0,13.0,14.0,health_beauty,3702.0,SP,-23.515978,-46.547372,5.0,2018-07-12,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113137,5bb40f4d5a4371842ecc05e3b4475dc1,7412,aruja,SP,-23.155308,-47.084074,dddc545a77663ef26d16778d03ef3bcd,delivered,2018-02-06 11:34:34,2018-02-06 11:49:49,2018-02-06 21:42:29,2018-02-10 14:33:09,2018-03-09 00:00:00,1.0,2018-02-12 11:49:49,549.00,18.59,2.0,481.0,18.0,13.0,15.0,watches_gifts,88370.0,SC,-26.890666,-48.655328,2.0,2018-02-11,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113172,535a05f4c66c1ebb2b8c6a537a7f2149,86996,sa

In [144]:
#Percentage of rows with missing values
print(df.isna().any(axis=1).sum()/len(df))
#Only 4.2%
print(len(df))

#Dropping fields with nas
df = df.dropna()

print(len(df))

df.head()

0.042010111001208925
109188
104601


,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,8683.0,54.0,64.0,31.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,10150.0,89.0,15.0,40.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,8267.0,52.0,52.0,17.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,12160.0,56.0,51.0,28.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,5200.0,45.0,15.0,35.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0


In [145]:
pd.options.display.max_columns = 100
df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,8683.0,54.0,64.0,31.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,10150.0,89.0,15.0,40.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,8267.0,52.0,52.0,17.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,12160.0,56.0,51.0,28.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,5200.0,45.0,15.0,35.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0


In [146]:
#Removing product weight, height and length information :
df.drop(columns={"product_weight_g","product_length_cm","product_height_cm","product_width_cm"},inplace=True)

#Turning the datetime columns into datetime
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df["order_approved_at"] = pd.to_datetime(df["order_approved_at"])
df["order_delivered_customer_date"] = pd.to_datetime(df["order_delivered_customer_date"])
df["order_estimated_delivery_date"] = pd.to_datetime(df["order_estimated_delivery_date"])
df["shipping_limit_date"] = pd.to_datetime(df["shipping_limit_date"])

In [147]:
#Checking for outliers

df.describe()
      

,customer_zip_code_prefix,customer_lat,customer_long,order_item_id,price,freight_value,product_photos_qty,seller_zip_code_prefix,seller_lat,seller_long,review_score,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher
count,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.000000,104601.0,104601.000000,104601.000000,104601.0,104601.000000,104601.000000,104601.000000,104601.0,104601.000000
mean,35134.118584,-21.255868,-46.234729,1.197551,119.823533,19.954127,2.210380,24558.623283,-22.791640,-47.248396,4.084771,0.421459,0.121366,1.005918,1.0,3.038403,179.255839,0.0,0.209166,0.765012,0.014981,0.0,0.010841
std,29936.676973,5.544591,4.024424,0.697262,179.829678,15.726098,1.722532,27705.306559,2.714594,2.346642,1.344000,0.496864,0.326992,0.077443,0.0,2.806137,270.405112,0.0,0.406715,0.423993,0.121476,0.0,0.103556
min,1003.000000,-33.689948,-72.668881,1.000000,0.850000,0.000000,1.000000,1001.000000,-32.079231,-63.893565,1.000000,0.000000,0.000000,1.000000,1.0,1.000000,10.070000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
25%,11089.000000,-23.592896,-48.125102,1.000000,39.900000,13.080000,1.000000,6429.000000,-23.609430,-48.829744,4.000000,0.000000,0.000000,1.000000,1.0,1.000000,65.710000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
50%,24240.000000,-22.930175,-46.636773,1.000000,74.900000,16.270000,1.000000,13660.000000,-23.418614,-46.755753,5.000000,0.000000,0.000000,1.000000,1.0,2.000000,114.330000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
75%,59069.000000,-20.179524,-43.705683,1.000000,134.900000,21.150000,3.000000,28615.000000,-21.757321,-46.522287,5.000000,1.000000,0.000000,1.000000,1.0,4.000000,195.420000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
max,99980.000000,42.184003,-8.723762,21.000000,6735.000000,409.680000,20.000000,99730.000000,-2.501242,-34.855616,5.000000,3.000000,2.000000,3.000000,1.0,24.000000,13664.080000,0.0,1.000000,1.000000,1.000000,0.0,1.000000


### 2.1 Feature engineering

In [148]:
from geopy import distance

#Creating distance between seller and customer
df["distance"] = df.apply(lambda row: distance.distance((row["seller_lat"],row["seller_long"]),
                                                        (row["customer_lat"], row["customer_long"])).km, axis=1)

df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher,distance
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0,347.500526
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0,412.888623
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0,30.001582
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0,18.599349
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0,219.923220


In [151]:
#Creating the time_difference between purchase and approved order
df["order_approval_delay"] = df["order_approved_at"] - df["order_purchase_timestamp"]

df["order_approval_delay"] = df["order_approval_delay"].apply(lambda x: x.total_seconds())
df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher,distance,order_approval_delay
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0,347.500526,997.0
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0,412.888623,608.0
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0,30.001582,87085.0
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0,18.599349,4961.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0,219.923220,1119.0


In [154]:
df["order_shipping_delay"] = df["order_estimated_delivery_date"] - df["order_purchase_timestamp"]
#Extracting delay in day
df["order_shipping_delay"] = df["order_shipping_delay"].apply(lambda x: x/np.timedelta64(1, 'D'))

#Difference between actual delivery and review
df["review_delay"] = df["review_date"]-df["order_delivered_customer_date"]

df["review_delay"] = df["review_delay"].apply(lambda x: x/np.timedelta64(1, 'D'))
df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher,distance,order_approval_delay,order_shipping_delay,review_delay
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0,347.500526,997.0,19.371123,0.558623
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0,412.888623,608.0,24.133056,0.471308
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0,30.001582,87085.0,24.327951,0.250799
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0,18.599349,4961.0,27.328727,0.330266
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0,219.923220,1119.0,16.589236,0.127917


In [159]:
#Removing the seller location information that we will not use
df_cat1 = pd.crosstab(df["customer_id"], df["product_category_name"]).reset_index()
df_cat2 = pd.crosstab(df["customer_id"], df["order_status"]).reset_index()

df_cat = pd.merge(df_cat1, df_cat2, how="inner", on="customer_id")

df_cat.head()

,customer_id,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,books_imported,books_technical,cds_dvds_musicals,christmas_supplies,cine_photo,computers,computers_accessories,consoles_games,construction_tools_construction,construction_tools_lights,construction_tools_safety,cool_stuff,costruction_tools_garden,costruction_tools_tools,diapers_and_hygiene,drinks,dvds_blu_ray,electronics,fashio_female_clothing,fashion_bags_accessories,fashion_childrens_clothes,fashion_male_clothing,fashion_shoes,fashion_sport,fashion_underwear_beach,fixed_telephony,flowers,food,food_drink,furniture_bedroom,furniture_decor,furniture_living_room,furniture_mattress_and_upholstery,garden_tools,health_beauty,home_appliances,home_confort,home_construction,housewares,industry_commerce_and_business,kitchen_and_food_preparation_appliances,kitchen_dining_laundry_garden_furniture,la_cuisine,luggage_accessories,market_place,music,musical_instruments,office_furniture,party_supplies,perfumery,pet_shop,security_and_services,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,watches_gifts,canceled,delivered
0,00012a2ce6f8dcda20d059ce98491703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,000161a058600d5901f007fab4c27140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0001fd6190edaaf884bcaf3d49edf079,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0002414f95344307404f0ace7a26f1d5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,000379cdec625522490c315e70c7a9fb,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0004164d20a9e969af783496f3408652,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
6,000419c5494106c306a97b5635748086,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,00046a560d407e99b969756e0b10f282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
8,00050bf6e01e69d5c0fd612f1bcfb69c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,000598caf2ef4117407665ac33275130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [160]:
df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_long,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,product_photos_qty,product_category_name,seller_zip_code_prefix,seller_state,seller_lat,seller_long,review_score,review_date,review_ncomments,review_ntitle,n_reviews,payment_sequential,payment_installments,payment_value,payment_sequential_flag,boleto,credit_card,debit_card,not_defined,voucher,distance,order_approval_delay,order_shipping_delay,review_delay
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1.0,2017-05-22 15:22:12,124.99,21.88,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,4.0,2017-05-26,0.0,0.0,1.0,1.0,2.0,146.87,0.0,0.0,1.0,0.0,0.0,0.0,347.500526,997.0,19.371123,0.558623
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP,-23.727992,-46.542848,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1.0,2018-01-18 20:58:32,289.00,46.48,3.0,housewares,88303.0,SC,-26.912574,-48.673980,5.0,2018-01-30,0.0,0.0,1.0,1.0,8.0,335.48,0.0,0.0,1.0,0.0,0.0,0.0,412.888623,608.0,24.133056,0.471308
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.531642,-46.656289,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1.0,2018-06-05 16:19:10,139.94,17.79,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-06-15,0.0,0.0,1.0,1.0,7.0,157.73,0.0,0.0,1.0,0.0,0.0,0.0,30.001582,87085.0,24.327951,0.250799
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.499702,-46.185233,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1.0,2018-03-27 16:31:16,149.94,23.36,1.0,office_furniture,8577.0,SP,-23.486111,-46.366721,5.0,2018-03-29,0.0,0.0,1.0,1.0,1.0,173.30,0.0,0.0,1.0,0.0,0.0,0.0,18.599349,4961.0,27.328727,0.330266
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP,-22.975100,-47.142925,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1.0,2018-07-31 10:10:09,230.00,22.25,1.0,home_confort,14940.0,SP,-21.757321,-48.829744,5.0,2018-08-10,1.0,1.0,1.0,1.0,8.0,252.25,0.0,0.0,1.0,0.0,0.0,0.0,219.923220,1119.0,16.589236,0.127917


In [ ]:
df_num = df.groupby("customer_id").agg(
    customer_state = ("customer_state", "mode"), earliest_order = ("order_purchase_timestamp","min"),
    latest_order = ("order_purchase_timestamp", "max"), avg_item_price = ("price", "mean"),
    avg_photos('product_photos_qty', 'mean'), avg_review_score = ('review_score', 'mean'),
    total_comments = ('')

)